In [2]:
import pandas as pd
import numpy as np

from lightgbm import LGBMClassifier
import category_encoders as ce
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [41]:
path = "./"
train = pd.read_csv(path+'train.csv', index_col=0)
test = pd.read_csv(path+'test.csv', index_col=0)
pseudo_label = pd.read_csv(path+'pseudo-labels.csv', index_col=0)
submission = pd.read_csv(path+'sample_submission.csv')
pseudo_label

,Survived
PassengerId,
100000,0
100001,1
100002,1
100003,0
100004,1
...,...
199995,1
199996,0
199997,0


In [42]:
test['Survived']  = pseudo_label
test

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
PassengerId,,,,,,,,,,,
100000,3,"Holliday, Daniel",male,19.0,0,0,24745,63.01,NaN,S,0
100001,3,"Nguyen, Lorraine",female,53.0,0,0,13264,5.81,NaN,S,1
100002,1,"Harris, Heather",female,19.0,0,0,25990,38.91,B15315,C,1
100003,2,"Larsen, Eric",male,25.0,0,0,314011,12.93,NaN,S,0
100004,1,"Cleary, Sarah",female,17.0,0,2,26203,26.89,B22515,C,1
...,...,...,...,...,...,...,...,...,...,...,...
199995,3,"Cash, Cheryle",female,27.0,0,0,7686,10.12,NaN,Q,1
199996,1,"Brown, Howard",male,59.0,1,0,13004,68.31,NaN,S,0
199997,3,"Lightfoot, Cameron",male,47.0,0,0,4383317,10.87,NaN,S,0


In [21]:
# Calcule SameFirstName

train['FirstName'] = train['Name'].apply(lambda x:x.split(', ')[0])
train['n'] = 1
gb = train.groupby('FirstName')
df_names = gb['n'].sum()
train['SameFirstName'] = train['FirstName'].apply(lambda x:df_names[x])

test['FirstName'] = test['Name'].apply(lambda x:x.split(', ')[0])
test['n'] = 1
gb = test.groupby('FirstName')
df_names = gb['n'].sum()
test['SameFirstName'] = test['FirstName'].apply(lambda x:df_names[x])

# To preprocess

data = pd.concat([train, test], axis=0)

# Before fill missing
data['AnyMissing'] = np.where(data.isnull().any(axis=1) == True, 1, 0)

# Family
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
data['IsAlone'] = np.where(data['FamilySize'] <= 1, 1, 0)

# Cabin
data['Has_Cabin'] = data["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
data['Cabin'] = data['Cabin'].fillna('X').map(lambda x: x[0].strip())
cabin_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5,
             'F': 6, 'G': 7, 'T': 1, 'X': 8}
data['Cabin'] = data['Cabin'].str[0].fillna('X').replace(cabin_map)

# Embarked
#map_Embarked = train.Embarked.mode().item()
data['Embarked'] = data['Embarked'].fillna("No")
conditions = [
    (data['Embarked']=="S"),
    (data['Embarked']=="Q"),
    (data['Embarked']=="C"),
    (data['Embarked']=="No")
]
choices = [0, 1, 2, -1]
data["Embarked"] = np.select(conditions, choices)
data['Embarked'] = data['Embarked'].astype(int)

# Name
data['SecondName'] = data.Name.str.split(', ', 1, expand=True)[1] # to try
data['IsFirstNameDublicated'] = np.where(data.FirstName.duplicated(), 1, 0)

# Fare
data['Fare'] = data['Fare'].fillna(train['Fare'].median())
# train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
# [(0.679, 10.04] < (10.04, 24.46] < (24.46, 33.5] < (33.5, 744.66]]
# From original Titanic:
conditions = [
    (data['Fare'] <= 7.91),
    ((data['Fare'] > 7.91) & (data['Fare'] <= 14.454)),
    ((data['Fare'] > 14.454) & (data['Fare'] <= 31)),
    (data['Fare'] > 31)
]

choices = [0, 1, 2, 3]
data["Fare"] = np.select(conditions, choices)
data['Fare'] = data['Fare'].astype(int)

# Fix Ticket
# data['TicketNum'] = data.Ticket.str.extract(r'(\d+)').\
#                     astype('float64', copy=False) # to_try
data['Ticket'] = data.Ticket.str.replace('\.','', regex=True).\
                    str.replace('(\d+)', '', regex=True).\
                    str.replace(' ', '', regex=True).\
                    replace(r'^\s*$', 'X', regex=True).\
                    fillna('X')

#data['Ticket'] = data['Ticket'].astype('category').cat.codes # to_try

# Age 
conditions = [
    ((data.Sex=="female")&(data.Pclass==1)&(data.Age.isnull())),
    ((data.Sex=="male")&(data.Pclass==1)&(data.Age.isnull())),
    ((data.Sex=="female")&(data.Pclass==2)&(data.Age.isnull())),
    ((data.Sex=="male")&(data.Pclass==2)&(data.Age.isnull())),
    ((data.Sex=="female")&(data.Pclass==3)&(data.Age.isnull())),
    ((data.Sex=="male")&(data.Pclass==3)&(data.Age.isnull()))
]
choices = data[['Age', 'Pclass', 'Sex']].\
            dropna().\
            groupby(['Pclass', 'Sex']).\
            mean()['Age']

data["Age"] = np.select(conditions, choices)

conditions = [
    (data['Age'].le(16)),
    (data['Age'].gt(16) & data['Age'].le(32)),
    (data['Age'].gt(32) & data['Age'].le(48)),
    (data['Age'].gt(48) & data['Age'].le(64)),
    (data['Age'].gt(64))
]
choices = [0, 1, 2, 3, 4]

data["Age"] = np.select(conditions, choices)

# Sex
data['Sex'] = np.where(data['Sex']=='male', 1, 0)

# Drop columns
data = data.drop(['Name', 'n'], axis = 1)

# Transform object to category
for col in data.columns[data.dtypes=='object'].tolist():
   data.loc[:,col] = data.loc[:,col].astype('category')

In [43]:
# better encoding
all_df = pd.concat([train, test], axis=0)
# Age fillna with mean age for each class
all_df['Age'] = all_df['Age'].fillna(all_df['Age'].mean())

# Cabin, fillna with 'X' and take first letter
all_df['Cabin'] = all_df['Cabin'].fillna('X').map(lambda x: x[0].strip())

# Ticket, fillna with 'X', split string and take first split 
all_df['Ticket'] = all_df['Ticket'].fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')

# Fare, fillna with mean value
fare_map = all_df[['Fare', 'Pclass']].dropna().groupby('Pclass').median().to_dict()
all_df['Fare'] = all_df['Fare'].fillna(all_df['Pclass'].map(fare_map['Fare']))
all_df['Fare'] = np.log1p(all_df['Fare'])

# Embarked, fillna with 'X' value
all_df['Embarked'] = all_df['Embarked'].fillna('X')

# Name, take only surnames
all_df['Name'] = all_df['Name'].map(lambda x: x.split(',')[0])

# encoding
label_cols = ['Name', 'Ticket', 'Sex']
onehot_cols = ['Cabin', 'Embarked']
numerical_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

def label_encoder(c):
    le = LabelEncoder()
    return le.fit_transform(c)

scaler = StandardScaler()

onehot_encoded_df = pd.get_dummies(all_df[onehot_cols])
label_encoded_df = all_df[label_cols].apply(label_encoder)
numerical_df = pd.DataFrame(scaler.fit_transform(all_df[numerical_cols]), columns=numerical_cols)
target_df = all_df[target]

all_df = pd.concat([numerical_df, label_encoded_df, onehot_encoded_df, target_df], axis=1)
data = all_df

In [44]:
# Splitting into train and test
train = data.iloc[:train.shape[0]]
test = data.iloc[train.shape[0]:].drop(columns=['Survived'])

In [46]:
train

,Pclass,Age,SibSp,Parch,Fare,Name,Ticket,Sex,Cabin_A,Cabin_B,...,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_X,Embarked_C,Embarked_Q,Embarked_S,Embarked_X,Survived
0,-1.425730,-4.307127e-16,1.901268,-0.505478,0.134351,17441,49,1,0,0,...,0,0,0,0,0,0,0,1,0,1
1,0.877699,-4.307127e-16,-0.539572,-0.505478,-0.533837,3063,49,1,0,0,...,0,0,0,0,1,0,0,1,0,0
2,0.877699,-2.069149e+00,0.680848,1.628715,1.070483,17798,14,1,0,0,...,0,0,0,0,1,0,0,1,0,0
3,0.877699,-9.374220e-01,-0.539572,-0.505478,-0.555506,12742,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
4,0.877699,-5.737175e-01,-0.539572,-0.505478,-1.023540,2335,49,1,0,0,...,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-0.274016,1.669127e+00,-0.539572,-0.505478,-0.434567,1590,21,0,0,0,...,0,0,0,0,0,1,0,0,0,1
99996,-0.274016,1.911597e+00,-0.539572,-0.505478,-0.698959,2992,49,1,0,0,...,0,0,0,0,1,0,0,1,0,0
99997,0.877699,1.536915e-01,-0.539572,-0.505478,-0.802137,4219,49,1,0,0,...,0,0,0,0,1,0,0,1,0,0
99998,0.877699,1.002335e+00,-0.539572,0.561618,0.259408,3941,49,1,0,0,...,0,0,0,0,1,0,0,1,0,0


In [48]:
lab_cols = ['Pclass','Age', 'Ticket', 'Fare', 'Cabin', 'Embarked']
target = 'Survived'

# features_selected = ['Pclass', 'Sex', 'Age','Embarked','Parch','SibSp','Fare','Cabin','Ticket','SameFirstName']
features_selected = ['Pclass', 'Sex', 'Age','Embarked','Parch','SibSp','Fare','Cabin','Ticket', 'Name']

X = data.drop(target, axis=1)
# X = X[features_selected]
y = data[target]

# test = test[features_selected]

In [49]:
def kfold_prediction(X, y, X_test, K, params):

    yp = pd.DataFrame()
    trs = []
    acc_trs = []
    
    kf = StratifiedKFold(n_splits=K, shuffle=True, random_state=314)
    
    for i, (train_idx, test_idx) in enumerate(kf.split(X, y)):
        print(f"\n FOLD {i} ...")
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[test_idx]
        y_val = y.iloc[test_idx]
        
        clf = LGBMClassifier(**params)
        
        model_fit = clf.fit(X_train,y_train,
                            eval_set=[(X_train, y_train), (X_val, y_val)])
        
        yp_val = model_fit.predict_proba(X_val)[:, 1]
        acc = accuracy_score(y_val, np.where(yp_val>0.5, 1, 0))
        print(f"- Accuracy before : {acc} ...")
        
        # Moving threshold
        thresholds = np.arange(0.0, 1.0, 0.01)
        accuracy_scores = []
        for thresh in thresholds:
            accuracy_scores.append(
                accuracy_score(y_val, [1 if m>thresh else 0 for m in yp_val]))

        accuracies = np.array(accuracy_scores)
        max_accuracy = accuracies.max() 
        max_accuracy_threshold =  thresholds[accuracies.argmax()] 
        trs = trs + [max_accuracy_threshold]
        
        print("- Max accuracy threshold: "+str(max_accuracy_threshold))
        
        acc = accuracy_score(y_val, 
                             np.where(yp_val>max_accuracy_threshold, 1, 0)) 
        acc_trs = acc_trs + [acc]
        print(f"- Accuracy after: {acc} !")
        
        yp_test = model_fit.predict_proba(X_test)[:, 1]
        yp_fold = pd.DataFrame({
            'fold'+str(i): np.where(yp_test>max_accuracy_threshold, 1, 0)})
        
        yp = pd.concat([yp, yp_fold], axis=1)
    
    return yp, trs, acc_trs

In [57]:
yp,yp.columns.tolist()

(       fold0  fold1  fold2  fold3  fold4
 0          0      0      0      0      0
 1          1      1      1      1      1
 2          1      1      1      1      1
 3          0      0      0      0      0
 4          1      1      1      1      1
 ...      ...    ...    ...    ...    ...
 99995      1      1      1      1      1
 99996      0      0      0      0      0
 99997      0      0      0      0      0
 99998      1      1      1      1      1
 99999      1      1      1      1      1
 
 [100000 rows x 5 columns],
 ['fold0', 'fold1', 'fold2', 'fold3', 'fold4'])

In [53]:
params = {'reg_alpha': 0.03853704528347309,
 'reg_lambda': 0.785624238068156,
 'num_leaves': 286,
 'learning_rate': 0.05335675750476966,
 'max_depth': 9,
 'n_estimators': 876,
 'min_child_samples': 56,
 'min_child_weight': 0.0005186164757607733,
 'subsample': 0.6730566683826382,
 'colsample_bytree': 0.48885018746352044,
 'cat_smooth': 5.281189960221718}
yp, trs, acc = kfold_prediction(X, y, test, 5,  params)


 FOLD 0 ...
[1]	training's binary_logloss: 0.637244	valid_1's binary_logloss: 0.637228
[2]	training's binary_logloss: 0.62309	valid_1's binary_logloss: 0.623401
[3]	training's binary_logloss: 0.611735	valid_1's binary_logloss: 0.612124
[4]	training's binary_logloss: 0.600083	valid_1's binary_logloss: 0.600644
[5]	training's binary_logloss: 0.577589	valid_1's binary_logloss: 0.578132
[6]	training's binary_logloss: 0.568589	valid_1's binary_logloss: 0.569108
[7]	training's binary_logloss: 0.561096	valid_1's binary_logloss: 0.561945
[8]	training's binary_logloss: 0.552566	valid_1's binary_logloss: 0.553688
[9]	training's binary_logloss: 0.534812	valid_1's binary_logloss: 0.535906
[10]	training's binary_logloss: 0.528336	valid_1's binary_logloss: 0.52966
[11]	training's binary_logloss: 0.523068	valid_1's binary_logloss: 0.524585
[12]	training's binary_logloss: 0.516515	valid_1's binary_logloss: 0.518277
[13]	training's binary_logloss: 0.511799	valid_1's binary_logloss: 0.513484
[14]	train

[123]	training's binary_logloss: 0.286526	valid_1's binary_logloss: 0.300967
[124]	training's binary_logloss: 0.286215	valid_1's binary_logloss: 0.300762
[125]	training's binary_logloss: 0.286128	valid_1's binary_logloss: 0.300751
[126]	training's binary_logloss: 0.285653	valid_1's binary_logloss: 0.30051
[127]	training's binary_logloss: 0.285606	valid_1's binary_logloss: 0.300495
[128]	training's binary_logloss: 0.285338	valid_1's binary_logloss: 0.300471
[129]	training's binary_logloss: 0.284839	valid_1's binary_logloss: 0.30014
[130]	training's binary_logloss: 0.284528	valid_1's binary_logloss: 0.299917
[131]	training's binary_logloss: 0.284395	valid_1's binary_logloss: 0.299881
[132]	training's binary_logloss: 0.284292	valid_1's binary_logloss: 0.299888
[133]	training's binary_logloss: 0.284243	valid_1's binary_logloss: 0.299861
[134]	training's binary_logloss: 0.284139	valid_1's binary_logloss: 0.299847
[135]	training's binary_logloss: 0.28383	valid_1's binary_logloss: 0.299647
[1

[267]	training's binary_logloss: 0.268074	valid_1's binary_logloss: 0.295745
[268]	training's binary_logloss: 0.267988	valid_1's binary_logloss: 0.295704
[269]	training's binary_logloss: 0.267906	valid_1's binary_logloss: 0.295676
[270]	training's binary_logloss: 0.267781	valid_1's binary_logloss: 0.295668
[271]	training's binary_logloss: 0.267729	valid_1's binary_logloss: 0.295675
[272]	training's binary_logloss: 0.267677	valid_1's binary_logloss: 0.295678
[273]	training's binary_logloss: 0.267602	valid_1's binary_logloss: 0.295677
[274]	training's binary_logloss: 0.267595	valid_1's binary_logloss: 0.295678
[275]	training's binary_logloss: 0.2675	valid_1's binary_logloss: 0.295658
[276]	training's binary_logloss: 0.267476	valid_1's binary_logloss: 0.295654
[277]	training's binary_logloss: 0.267383	valid_1's binary_logloss: 0.295665
[278]	training's binary_logloss: 0.267359	valid_1's binary_logloss: 0.29567
[279]	training's binary_logloss: 0.267334	valid_1's binary_logloss: 0.295672
[2

[389]	training's binary_logloss: 0.260988	valid_1's binary_logloss: 0.295954
[390]	training's binary_logloss: 0.260952	valid_1's binary_logloss: 0.295937
[391]	training's binary_logloss: 0.260907	valid_1's binary_logloss: 0.295951
[392]	training's binary_logloss: 0.260884	valid_1's binary_logloss: 0.295957
[393]	training's binary_logloss: 0.260815	valid_1's binary_logloss: 0.295976
[394]	training's binary_logloss: 0.260781	valid_1's binary_logloss: 0.295961
[395]	training's binary_logloss: 0.260737	valid_1's binary_logloss: 0.295955
[396]	training's binary_logloss: 0.260663	valid_1's binary_logloss: 0.29594
[397]	training's binary_logloss: 0.260629	valid_1's binary_logloss: 0.295947
[398]	training's binary_logloss: 0.26056	valid_1's binary_logloss: 0.295962
[399]	training's binary_logloss: 0.260527	valid_1's binary_logloss: 0.295946
[400]	training's binary_logloss: 0.260477	valid_1's binary_logloss: 0.295958
[401]	training's binary_logloss: 0.260462	valid_1's binary_logloss: 0.295964
[

[513]	training's binary_logloss: 0.255482	valid_1's binary_logloss: 0.296509
[514]	training's binary_logloss: 0.255435	valid_1's binary_logloss: 0.296507
[515]	training's binary_logloss: 0.255421	valid_1's binary_logloss: 0.296511
[516]	training's binary_logloss: 0.255368	valid_1's binary_logloss: 0.296511
[517]	training's binary_logloss: 0.25529	valid_1's binary_logloss: 0.296537
[518]	training's binary_logloss: 0.255251	valid_1's binary_logloss: 0.296546
[519]	training's binary_logloss: 0.255169	valid_1's binary_logloss: 0.296558
[520]	training's binary_logloss: 0.255148	valid_1's binary_logloss: 0.296557
[521]	training's binary_logloss: 0.255036	valid_1's binary_logloss: 0.296572
[522]	training's binary_logloss: 0.254942	valid_1's binary_logloss: 0.296582
[523]	training's binary_logloss: 0.25486	valid_1's binary_logloss: 0.296596
[524]	training's binary_logloss: 0.254819	valid_1's binary_logloss: 0.296601
[525]	training's binary_logloss: 0.25475	valid_1's binary_logloss: 0.296613
[5

...

[249]	training's binary_logloss: 0.268226	valid_1's binary_logloss: 0.299462
[250]	training's binary_logloss: 0.268165	valid_1's binary_logloss: 0.299474
[251]	training's binary_logloss: 0.267991	valid_1's binary_logloss: 0.299494
[252]	training's binary_logloss: 0.267965	valid_1's binary_logloss: 0.299494
[253]	training's binary_logloss: 0.267923	valid_1's binary_logloss: 0.299502
[254]	training's binary_logloss: 0.267831	valid_1's binary_logloss: 0.29951
[255]	training's binary_logloss: 0.26774	valid_1's binary_logloss: 0.299516
[256]	training's binary_logloss: 0.267653	valid_1's binary_logloss: 0.2995
[257]	training's binary_logloss: 0.267539	valid_1's binary_logloss: 0.299503
[258]	training's binary_logloss: 0.267465	valid_1's binary_logloss: 0.299487
[259]	training's binary_logloss: 0.267322	valid_1's binary_logloss: 0.299487
[260]	training's binary_logloss: 0.267293	valid_1's binary_logloss: 0.299497
[261]	training's binary_logloss: 0.267253	valid_1's binary_logloss: 0.299503
[26

[394]	training's binary_logloss: 0.259909	valid_1's binary_logloss: 0.299723
[395]	training's binary_logloss: 0.259855	valid_1's binary_logloss: 0.299732
[396]	training's binary_logloss: 0.259784	valid_1's binary_logloss: 0.299725
[397]	training's binary_logloss: 0.259769	valid_1's binary_logloss: 0.29973
[398]	training's binary_logloss: 0.259735	valid_1's binary_logloss: 0.29974
[399]	training's binary_logloss: 0.259722	valid_1's binary_logloss: 0.299738
[400]	training's binary_logloss: 0.259676	valid_1's binary_logloss: 0.299758
[401]	training's binary_logloss: 0.259664	valid_1's binary_logloss: 0.299759
[402]	training's binary_logloss: 0.259633	valid_1's binary_logloss: 0.299748
[403]	training's binary_logloss: 0.259541	valid_1's binary_logloss: 0.299742
[404]	training's binary_logloss: 0.259468	valid_1's binary_logloss: 0.299707
[405]	training's binary_logloss: 0.259434	valid_1's binary_logloss: 0.299707
[406]	training's binary_logloss: 0.259408	valid_1's binary_logloss: 0.299716
[

[544]	training's binary_logloss: 0.253368	valid_1's binary_logloss: 0.300497
[545]	training's binary_logloss: 0.253355	valid_1's binary_logloss: 0.300495
[546]	training's binary_logloss: 0.25333	valid_1's binary_logloss: 0.300497
[547]	training's binary_logloss: 0.253294	valid_1's binary_logloss: 0.300489
[548]	training's binary_logloss: 0.253265	valid_1's binary_logloss: 0.300497
[549]	training's binary_logloss: 0.253184	valid_1's binary_logloss: 0.300512
[550]	training's binary_logloss: 0.253136	valid_1's binary_logloss: 0.300515
[551]	training's binary_logloss: 0.253075	valid_1's binary_logloss: 0.30053
[552]	training's binary_logloss: 0.253056	valid_1's binary_logloss: 0.300534
[553]	training's binary_logloss: 0.253021	valid_1's binary_logloss: 0.300549
[554]	training's binary_logloss: 0.252944	valid_1's binary_logloss: 0.300554
[555]	training's binary_logloss: 0.25293	valid_1's binary_logloss: 0.300554
[556]	training's binary_logloss: 0.252895	valid_1's binary_logloss: 0.300574
[5

[671]	training's binary_logloss: 0.248061	valid_1's binary_logloss: 0.30117
[672]	training's binary_logloss: 0.24804	valid_1's binary_logloss: 0.301158
[673]	training's binary_logloss: 0.248011	valid_1's binary_logloss: 0.301158
[674]	training's binary_logloss: 0.247998	valid_1's binary_logloss: 0.301163
[675]	training's binary_logloss: 0.24797	valid_1's binary_logloss: 0.301175
[676]	training's binary_logloss: 0.247959	valid_1's binary_logloss: 0.301173
[677]	training's binary_logloss: 0.247918	valid_1's binary_logloss: 0.301193
[678]	training's binary_logloss: 0.247881	valid_1's binary_logloss: 0.301203
[679]	training's binary_logloss: 0.247838	valid_1's binary_logloss: 0.301224
[680]	training's binary_logloss: 0.247794	valid_1's binary_logloss: 0.301244
[681]	training's binary_logloss: 0.247693	valid_1's binary_logloss: 0.30126
[682]	training's binary_logloss: 0.247627	valid_1's binary_logloss: 0.301279
[683]	training's binary_logloss: 0.2476	valid_1's binary_logloss: 0.301288
[684]

[803]	training's binary_logloss: 0.242529	valid_1's binary_logloss: 0.302158
[804]	training's binary_logloss: 0.242474	valid_1's binary_logloss: 0.302178
[805]	training's binary_logloss: 0.242449	valid_1's binary_logloss: 0.3022
[806]	training's binary_logloss: 0.242429	valid_1's binary_logloss: 0.302212
[807]	training's binary_logloss: 0.24235	valid_1's binary_logloss: 0.302246
[808]	training's binary_logloss: 0.242322	valid_1's binary_logloss: 0.302249
[809]	training's binary_logloss: 0.242275	valid_1's binary_logloss: 0.302267
[810]	training's binary_logloss: 0.242241	valid_1's binary_logloss: 0.30228
[811]	training's binary_logloss: 0.242221	valid_1's binary_logloss: 0.302294
[812]	training's binary_logloss: 0.242166	valid_1's binary_logloss: 0.302327
[813]	training's binary_logloss: 0.242143	valid_1's binary_logloss: 0.302336
[814]	training's binary_logloss: 0.242072	valid_1's binary_logloss: 0.302351
[815]	training's binary_logloss: 0.242036	valid_1's binary_logloss: 0.30236
[816

In [54]:
print('Model with train + pseudo train')
print("Final mean and std accuracy: ", np.mean(acc), round(np.std(acc), 5))
print("Final mean and std accuracy with Threshold: ", np.mean(trs), round(np.std(trs), 5))
# v2

Model with train + pseudo train
Final mean and std accuracy:  0.8857000000000002 0.00113
Final mean and std accuracy with Threshold:  0.484 0.02245


In [51]:
print('Model with train + pseudo train')
print("Final mean and std accuracy: ", np.mean(acc), round(np.std(acc), 5))
print("Final mean and std accuracy with Threshold: ", np.mean(trs), round(np.std(trs), 5))
#v1

Model with train + pseudo train
Final mean and std accuracy:  0.87538 0.00171
Final mean and std accuracy with Threshold:  0.47400000000000003 0.01356


In [3]:
def vote(r, columns):
    ones = 0
    zeros = 0
    for i in columns:
        if r[i]==0:
            zeros+=1
        else:
            ones+=1
    if ones>zeros:
        return 1
    else:
        return 0

In [55]:
submission_pseudo = pd.DataFrame({
    'PassengerId': test.index,
    'Survived':yp.apply(lambda x:vote(x, yp.columns.tolist()),axis=1)
})

submission_pseudo.to_csv('submission_pseudo_test_ohe.csv', index = False) # best 0.81730 LB

In [4]:
# voting with cat, more bagged models

lgbm_pred = pd.read_csv("submission_pseudo_test_ohe.csv")
cat_pred = pd.read_csv("cat_1.csv")
bag_pred = pd.read_csv("bag_1.csv")
lgbm_pred, cat_pred, bag_pred

(       PassengerId  Survived
 0           100000         0
 1           100001         1
 2           100002         1
 3           100003         0
 4           100004         1
 ...            ...       ...
 99995       199995         1
 99996       199996         0
 99997       199997         0
 99998       199998         1
 99999       199999         1
 
 [100000 rows x 2 columns],
        PassengerId  Survived
 0           100000         0
 1           100001         1
 2           100002         1
 3           100003         0
 4           100004         1
 ...            ...       ...
 99995       199995         1
 99996       199996         0
 99997       199997         0
 99998       199998         1
 99999       199999         1
 
 [100000 rows x 2 columns],
        PassengerId  Survived
 0           100000         0
 1           100001         1
 2           100002         1
 3           100003         0
 4           100004         1
 ...            ...       ...
 99995    

In [7]:
lgbm_pred["cat"] = cat_pred["Survived"]
lgbm_pred["bag"] = cat_pred["Survived"]
lgbm_pred

,PassengerId,Survived,cat,bag
0,100000,0,0,0
1,100001,1,1,1
2,100002,1,1,1
3,100003,0,0,0
4,100004,1,1,1
...,...,...,...,...
99995,199995,1,1,1
99996,199996,0,0,0
99997,199997,0,0,0
99998,199998,1,1,1


In [8]:
submission_voting = pd.DataFrame({
    'PassengerId': lgbm_pred["PassengerId"],
    'Survived':lgbm_pred.apply(lambda x:vote(x, ["Survived", "cat", "bag"]),axis=1)
})
submission_voting

,PassengerId,Survived
0,100000,0
1,100001,1
2,100002,1
3,100003,0
4,100004,1
...,...,...
99995,199995,1
99996,199996,0
99997,199997,0
99998,199998,1


In [14]:
(submission_voting["Survived"] == lgbm_pred["Survived"]).all()

False

In [15]:
submission_voting.to_csv('submission_voting.csv', index = False)